In [1]:
import pyNeuroChem as pyc
import hdnntools as hdt
import numpy as np
import time

from ase_interface import ANIENS
from ase_interface import ensemblemolecule

from ase.atoms import Atoms
from ase.optimize import BFGS, LBFGS

import pyaniasetools as pya

import batch_optimizers as opt

van Der Waals correction will be unavailable. Please install ased3


ModuleNotFoundError: No module named 'batch_optimizers'

In [ ]:
mol = hdt.readxyz2('/nh/nest/u/jsmith/Research/confs_test/capsaicin.xyz')
N = 1000

wkdir = '/home/jsmith48/scratch/ANI-2x_retrain/model1/'
cns = wkdir + 'rHCNOSFCl-4.6R_16-3.1A_a4-8.params'
sae = wkdir + 'sae_linfit.dat'
nnf = wkdir + 'train'
Nn = 5

In [3]:
Xl = np.stack([mol[0][0] for i in range(N)])
S = list(mol[1])
Xl.shape

(1000, 49, 3)

In [4]:
X = Xl+np.random.uniform(-0.15,0.15,size=Xl.shape)
X = np.asarray(X, dtype=np.float32)
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify1.xyz',X,S)

In [7]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [8]:
start_time = time.time()
for i,x in enumerate(X):
    mol = Atoms(S,x)
    mol.set_calculator(ANIENS(aens))
    
    dyn = LBFGS(mol, logfile='log.out')
    dyn.run(fmax=0.00018)
    print('Molecule',i,'complete')
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Molecule 0 complete
Molecule 1 complete
Molecule 2 complete
Molecule 3 complete
Molecule 4 complete
Molecule 5 complete
Molecule 6 complete


KeyboardInterrupt: 

In [ ]:
molopt = opt.moleculeOptimizer(cns, sae, nnf, Nn, 0)

In [ ]:
start_time = time.time()
X_it = molopt.optimizeGradientDescent(X,S,alpha=0.0004,convergence=1.0, printer=False)
X_it = molopt.optimizeConjugateGradient(X_it,S,alpha=0.0001,convergence=0.005, maxsteps=5000, printstep=100, printer=False)
print('[Complete Optimization Time:',time.time()-start_time,'s]')
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol1.xyz',np.array(X_it),S)

In [5]:
cnfopt = opt.conformerOptimizer(cns, sae, nnf, Nn, 0)

In [6]:
start_time = time.time()
X_it = cnfopt.optimizeConjugateGradient(X,S,alpha=0.0004,convergence=0.005, maxsteps=10000, printstep=100)
print('[Complete Optimization Time:',time.time()-start_time,'s]')
hdt.writexyzfile('/nh/nest/u/jsmith/Research/confs_test/verify_mol2.xyz',np.array(X_it),S)

--Optimizing all conformations--
  - 0000 ) 0 of 1000 10501.1807 374.1172 9.2442
  - 0100 ) 0 of 1000 60.3686 2.3999 0.0537
  - 0200 ) 0 of 1000 32.9174 1.4670 0.0352
  - 0300 ) 0 of 1000 19.5576 0.7793 0.0592
  - 0400 ) 0 of 1000 20.4024 0.7226 0.0759
  - 0500 ) 0 of 1000 28.0791 0.9551 0.0925
  - 0600 ) 0 of 1000 28.7705 1.0101 0.1248
  - 0700 ) 0 of 1000 30.2837 1.1434 0.1468
  - 0800 ) 0 of 1000 29.3476 1.1364 0.1624
  - 0900 ) 0 of 1000 30.9374 1.2121 0.1767
  - 1000 ) 0 of 1000 29.9665 1.1881 0.1850
  - 1100 ) 0 of 1000 30.0962 1.2099 0.1921
  - 1200 ) 1 of 1000 29.5662 1.1901 0.1953


KeyboardInterrupt: 